# Topological Data Analysis
Matias Ollu, Yuguang Yao

## Abstact
This repository aims at presenting our results to solve the problem of the project of topological data analysis in CSC_42021_EP - Conception et analyse d'algorithmes (2024-2025) of the école Polytechnique. We present an alogrithm that computes the simplexes and the filtration valuse of a given $\alpha$-complex or Čech-complex of an arbitary point-set. 

We devolepped an alogrithm that is general enough to calculate the circumcircle of $n$ points in $d$-dimension euclidean space.

## Environment
We use the following packages:

In [2]:
import random
import numpy as np
from itertools import combinations

## $\alpha$-complex and Čech-complex

(À completer, les définitions strict et les theoremes utilisées, réference: https://courses.cs.duke.edu/fall06/cps296.1/)

We will firstly define the key concepts and give the theorem that we are going to use. And we will forlulize our problems formly.

* Define 1: 

## Task 1

>Task 1. Implement the computation of the minimal enclosing ball (MEB)
>of a set of points using one (or more) of the algorithms from https:// en.
>wikipedia.org/ wiki/ LP-type_problem.

We defined a class of Sphere(center,radius), and used the alogithm of Weltz(LP-Problem) to solve the MEB quetion.

We solved solve for the circumcircle of $n$ points in $d$-dimension euclidean space.

The Weltz algorithm uses the circumcirle to find the MEB

In [3]:
# This class is used to represent a sphere.
class Sphere:
    def __init__(self, center, radius):
        self.center = np.array(center)
        self.radius = radius

    def contains(self, point):
        """Check if the sphere contains the given point."""
        return np.linalg.norm(self.center - np.array(point)) <= self.radius #+ 1e-9
    
    def contains_strict(self,point):
        """Check if the sphere contains the given point."""
        norme = np.linalg.norm(self.center - np.array(point))
        return not (np.isclose((norme),self.radius) or norme>self.radius+1e-9)
    
    def onradius(self,point):
        """Check if the point is on the sphere"""
        return np.isclose(np.linalg.norm(self.center - np.array(point)),self.radius)

Here, we find the equations allowing to compute the circumcircle of a given number of nppoints under the condition $n\leq d+1$.

In practice, this condition always holds as a simplex is only defined for maximum d points.


Let $P_i^n$ be the $i$-ième coordinate of point $P^n$ in a set of $N$-points in $d$-dimensions  satisfying $1 \leq n \leq N \leq d+1$. Let $c$ the center of the circumcircle be a weighted average of the $N$ points: $c=\sum_n k_n P^{n0}$. 

Then the center of the circumcile satisfies, for $n,m\in[1,n]\cap Z$:

$$|c-P^n|=|c-P^m| $$
$$\sum_i (c_i-P_i^n)^2-(n\to m)=0 $$
$$\sum_i (P_i^n)^2+2P_i^nc_i-(n\to m)=0 $$
$$\sum_i ((P_i^n)^2-(P_i^m)^2)=\sum_l (P_i^{l0}\sum_i 2(P_i^n-P_i^m)) k_l  $$
$$|P^n|-|P^m|=\sum_l (P_i^{l0}\sum_i 2(P_i^n-P_i^m)) k_l  .$$

In our implementation, we set $m=0$
$$|P^n|-|P^0|=\sum_l (2P_i^l\sum_i (P_i^n-P_i^0)) k_l  $$

$$A_{mn}=2P^{m0} \cdot P^{n0} $$
$$b_n=|P^n|-|P^0|.$$

This is a linear system with $n-1$ équations and $n-1$ variables. 

After having solved the equation, we compute  $c=\sum_n k_n P^{n0}$ as $c=k \cdot P+P^0$.

In [ ]:
def make_sphere_n_points(points):
    """
     Finds the minimal circumcircle in the d-dimensional space for n points (n <= d+1).
    
    When utilising this method, we assume that the points are not colinear and that the dimension is at least 2.

    Params :
        points: np.ndarray, shape (n, d)
    Returns :
        Sphere
    """

    nppoints = np.array(points, dtype=float)

    if len(points) > nppoints.shape[1] + 1:
        raise ValueError("Number of points must be less than or equal to the dimension of the space plus one.")

    if len(points) == 0:
        print("No points given in circumcircle calculation")
        raise None;
    if len(points) == 1:
        return Sphere(center=points[0], radius=0) 
    if len(points) == 2:
        return Sphere(center=(nppoints[0]+nppoints[1])/2, radius=np.linalg.norm(nppoints[0]-nppoints[1])/2)
    
    print("nppoints shape:", nppoints.shape)

    # Calcul de la matrice A et du vecteur b
    diffs = nppoints[1:] - nppoints[0]  # Différences (P^n - P^0) pour n = 1, ..., N-1
    print("diffs shape:", diffs.shape)

    A = 2 * np.dot(diffs, diffs.T)  # Matrice A
    b = np.sum(diffs ** 2, axis=1)  # Vecteur b

    # Résolution du système linéaire pour trouver les coefficients k
    k = np.linalg.solve(A, b)

    # Calculer le centre
    center = nppoints[0] + np.dot(k, diffs)

    # Calculer le rayon
    radius = np.linalg.norm(center - nppoints[0])

    # Retourner une instance de Sphere
    return Sphere(center=center, radius=radius)

In [5]:
def trivial(R):
    """Find the minimal sphere for 0, 1 or mores points."""
    if not R:
        return Sphere([0, 0, 0], 0)
    elif len(R) == 1:
        return Sphere(R[0], 0)
    elif len(R) >= 1:
        return make_sphere_n_points(R)

def welzl(P, R):
    """Recursive implementation of Welzl's algorithm for 3D."""
    if not P or len(R) == len(P[0])+   1 :
        return trivial(R)

    p = P.pop(random.randint(0, len(P) - 1))
    D = welzl(P, R)

    if D.contains(p):
        P.append(p)
        return D

    result = welzl(P, R + [p])
    P.append(p)
    return result

def minimal_enclosing_sphere(points):
    """Compute the minimal enclosing sphere for a set of points."""
    points = points[:]
    random.shuffle(points)
    return welzl(points, [])

In [6]:
def test_task1():
    """Test cases for minimal enclosing sphere."""
    # Test 1: Single point
    points = [(0, 0, 0)]
    sphere = minimal_enclosing_sphere(points)
    assert np.allclose(sphere.center, [0, 0, 0])
    assert np.isclose(sphere.radius, 0)
    print("Test 1.1 passed!")

    # Test 2: Two points
    points = [(0, 0, 0), (2, 0, 0)]
    sphere = minimal_enclosing_sphere(points)
    assert np.allclose(sphere.center, [1, 0, 0])
    assert np.isclose(sphere.radius, 1)
    print("Test 1.2 passed!")

    # Test 3: Three points
    points = [(-10, 0, 0), (10, 0, 0), (0, 1, 0)]
    sphere = minimal_enclosing_sphere(points)
    assert np.allclose(sphere.center, [0, 0, 0])
    assert np.isclose(sphere.radius, 10)
    print("Test 1.3 passed!")

    # Test 4: Four points
    points = [(5, 0, 1), (-1, -3, 4), (-1, -4, -3), (-1, 4, -3)]
    sphere =  minimal_enclosing_sphere(points)
    assert np.allclose(sphere.center, [0, 0, 0])
    assert np.isclose(sphere.radius, np.sqrt(26))
    print("Test 1.4 passed!")

    print("All test cases passed!")

test_task1()

Test 1.1 passed!
Test 1.2 passed!
Test 1.3 passed!
nppoints shape: (3, 3)
diffs shape: (2, 3)
nppoints shape: (3, 3)
diffs shape: (2, 3)
nppoints shape: (3, 3)
diffs shape: (2, 3)
nppoints shape: (3, 3)
diffs shape: (2, 3)
nppoints shape: (4, 3)
diffs shape: (3, 3)
Test 1.4 passed!
All test cases passed!


## Task 2

>Given a set of n points in $R^d$, implement a naive algorithm that enumerates the simplexes of $C^k$ and their filtration value

In [7]:

def task2(points,emu):
    """Compute the filtration value for the points in emu"""
    points_chosen = [points[i] for i in emu]
    filtration = minimal_enclosing_sphere(points_chosen).radius

    return filtration

def enum_simplex2(points):
    "Enumerates the simplexes and their filtration values"
    parties= [] #on fait la liste des sous ensembles de points:

    i, imax = 0, 2**len(points)-1 #We construct and iterator. The points in the subsets are the ones for which the corresponding bit is 1
    while i <= imax:
        s = []
        j, jmax = 0, len(points)-1
        while j <= jmax:
            if (i>>j)&1 == 1:
                s.append(points[j])
            j += 1
        parties.append(s)
        i += 1 

    #print the filtration values and subsets:
    for enum in parties:
        filtration = task2(points,enum)
        print(f"({enum}) -> {filtration}")

In [8]:
def test_task2():
    P = [(5, 0, 1), (-1, -3, 4), (-1, -4, -3), (-1, 4, -3)]

    test_cases = [
        ([0], 0),
        ([1], 0),
        ([2], 0),
        ([3], 0),
        ([2, 1], 3.53553),
        ([1, 0], 3.67425),
        ([3, 2], 4),
        ([2, 0], 4.12311),
        ([3, 0], 4.12311),
        ([2, 1, 0], 4.39525),
        ([3, 2, 0], 4.71495),
        ([3, 1], 4.94975),
        ([3, 2, 1], 5),
        ([3, 1, 0], 5.04975),
        ([3, 2, 1, 0], 5.09902),
    ]

    for enu, expected in test_cases:
        assert np.allclose(task2(P, enu), expected), f"Test({enu}) failed!"
        print(f"Test({enu}) passed!")


test_task2()

Test([0]) passed!
Test([1]) passed!
Test([2]) passed!
Test([3]) passed!
Test([2, 1]) passed!
Test([1, 0]) passed!
Test([3, 2]) passed!
Test([2, 0]) passed!
Test([3, 0]) passed!
nppoints shape: (3, 3)
diffs shape: (2, 3)
Test([2, 1, 0]) passed!
nppoints shape: (3, 3)
diffs shape: (2, 3)
Test([3, 2, 0]) passed!
Test([3, 1]) passed!
nppoints shape: (3, 3)
diffs shape: (2, 3)
Test([3, 2, 1]) passed!
nppoints shape: (3, 3)
diffs shape: (2, 3)
Test([3, 1, 0]) passed!
nppoints shape: (3, 3)
diffs shape: (2, 3)
nppoints shape: (3, 3)
diffs shape: (2, 3)
nppoints shape: (3, 3)
diffs shape: (2, 3)
nppoints shape: (3, 3)
diffs shape: (2, 3)
nppoints shape: (4, 3)
diffs shape: (3, 3)
Test([3, 2, 1, 0]) passed!


## Task 3

>Reuse the LP-type algorithm with new parameters in order to determine
if a simplex is in the α-complex and its filtration value. Note that this is less
standard than for the MEB, you need to explain how this new problem fits in
the framework.

Why problem fits in framework:


Why the solution for points on the frontier works

In [9]:
def enum3(points):
    """
    Generates a table where each line corresponds to the subsets of a certain size.
    The subsets are represented by their indices in the points
    """
    n = len(points)
    return [[list(comb) for comb in combinations(range(n), k)] for k in range(1, n + 1)]
  
def task3_mathias(points,l):
    """implement an algorithm that enumerates the simplexes and their filtration values."""

    emu = enum3(points)
    simplex = {tuple([i]): 0 for i in range(len(points))} #on initialise le premier simplexe
    #IsSimplex = {0:True} #dictionnaire indiquant si un sous ensemble forme un simplexe
    IsSimplex= [[0] * len(sublist) for sublist in emu] # le tableau de suivi: 0 si jamais vu, 1 si simplexe, 2 si pas un simplexe

    n=len(points)


    #Vestion 1 Mathias: pas optimale, on évite pas tj de calculer les simplexes qui en contiennent d'autres
    for i in range(1,len(emu)):
        for j in range(len(emu[i])):
            test = IsSimplex[i][j]
            if(test==0): #pas encore connu
                filtration = task2(points,emu[i][j])
                if (filtration > l): #pas un simplexe
                    IsSimplex[i][j] = 2
                    if(i<n-1): #on s'assure qu'on est pas à la dernière ligne
                        for k in range(n-i-1-j):
                         IsSimplex[i+1][j+k]=2 #les sous ensembles de taille supérieure qui contiennent emu[i,j] ne sont pas non plus des simplexes
                else:
                    IsSimplex[i][j] = 1
                    simplex[tuple(emu[i][j])]=filtration
            elif(test==1): #est un simplexe -> normalement impossible de revenir sur nos pas !
                raise RecursionError("l'ago revient sur ses pas !")
            #Sinon, test ==2 et ce n'est pas un simplexe. on passe au prochain. Pas besoin de tester cette éventualité
    
    for key, value in simplex.items():
        print(f"{key} -> {value}")
    
    return simplex   

def task3(points,l):
    enum = enum3(points)
    IsSimplex = {tuple([i]): 1 for i in range(len(points))}

    simplex = {tuple([i]): Sphere(points[i], 0) for i in range(len(points))} #on initialise le premier simplexe

    
    for i in range(1,len(enum)):
        for j in range(len(enum[i-1])):
            current_simplex = enum[i-1][j]

            for k in range(len(points)):

                pn =tuple(current_simplex + [k])

                if k in current_simplex:
                    IsSimplex[pn] = 0
                    break

                if pn in simplex:
                    break

                new_points = [points[idx] for idx in current_simplex] + [points[k]]

                MEB = minimal_enclosing_sphere(new_points)

                if MEB.radius < l:
                    simplex[pn] = MEB
                    IsSimplex[pn] = 1
                else:
                    IsSimplex[pn] = 0

    for key, value in simplex.items():
        print(f"{key} -> {value.radius}")

In [10]:
def test_task3():
    P = [(5, 0, 1), (-1, -3, 4), (-1, -4, -3), (-1, 4, -3)]
    for i in [0,4,5,100]:
        print(f"-------------------filtration={i}-------------------")
        task3(P, i)

test_task3()

-------------------filtration=0-------------------
nppoints shape: (3, 3)
diffs shape: (2, 3)
nppoints shape: (3, 3)
diffs shape: (2, 3)
nppoints shape: (3, 3)
diffs shape: (2, 3)
nppoints shape: (3, 3)
diffs shape: (2, 3)
nppoints shape: (3, 3)
diffs shape: (2, 3)
nppoints shape: (3, 3)
diffs shape: (2, 3)
nppoints shape: (3, 3)
diffs shape: (2, 3)
nppoints shape: (3, 3)
diffs shape: (2, 3)
nppoints shape: (4, 3)
diffs shape: (3, 3)
(0,) -> 0
(1,) -> 0
(2,) -> 0
(3,) -> 0
-------------------filtration=4-------------------
nppoints shape: (3, 3)
diffs shape: (2, 3)
nppoints shape: (3, 3)
diffs shape: (2, 3)
nppoints shape: (3, 3)
diffs shape: (2, 3)
nppoints shape: (3, 3)
diffs shape: (2, 3)
nppoints shape: (3, 3)
diffs shape: (2, 3)
nppoints shape: (3, 3)
diffs shape: (2, 3)
nppoints shape: (3, 3)
diffs shape: (2, 3)
nppoints shape: (3, 3)
diffs shape: (2, 3)
nppoints shape: (4, 3)
diffs shape: (3, 3)
(0,) -> 0
(1,) -> 0
(2,) -> 0
(3,) -> 0
(1, 0) -> 3.6742346141747673
(2, 1) -> 3.535

## Task 4

>Given a set P of n points in $R^d$, implement an algorithm that enumerates the simplexes of dimension at most k and filtration value at most l of the α-complex and their filtration values.


In [20]:
def Is_in_alpha_complex(P,R):
    """Check if the simplex R is in the alpha complex of P"""
    
    #On suppose que le simplex est un simplexe
    if not R :
        return True
    

    circum=make_sphere_n_points(R)
    
    for p in P:
       if circum.contains_strict(p):
           return False

    return True

def filtration_value(R):
    """Compute the filtration value of the simplex R"""
    return make_sphere_n_points(R)

def task4(points,R):
    """"Reuse the LP-type algorithm with new parameters in order to determine
if a simplex is in the α-complex and its filtration value. Note that this is less
standard than for the MEB, you need to explain how this new problem fits in
the framework."""
    return Is_in_alpha_complex(points,R),filtration_value(R).radius, filtration_value(R).center

In [22]:
def test_task4():
    P=[(0,5,0),(3,4,0),(-3,4,0)]
    R=[(0,5,0),(3,4,0),(-3,4,0)]
    print(f"---- Test for {P}")
    a= task4(P,R)
    print(f"Complex ? {a[0]}")
    print(f"filtration value: {a[1]}")
    print(f"center {a[2]}")

    P.append((0,0,4))
    print(f"---- Test for {P}")
    a= task4(P,R)
    print(f"Complex ? {a[0]}")
    print(f"filtration value: {a[1]}")
    print(f"center {a[2]}")


    P.append((0,0,-4))
    print(f"---- Test for {P}")
    a= task4(P,R)
    print(f"Complex ? {a[0]}")
    print(f"filtration value: {a[1]}")
    print(f"center {a[2]}")


test_task4()

---- Test for [(0, 5, 0), (3, 4, 0), (-3, 4, 0)]
nppoints shape: (3, 3)
diffs shape: (2, 3)
nppoints shape: (3, 3)
diffs shape: (2, 3)
nppoints shape: (3, 3)
diffs shape: (2, 3)
Complex ? True
filtration value: 5.000000000000001
center [ 4.4408921e-16 -8.8817842e-16  0.0000000e+00]
---- Test for [(0, 5, 0), (3, 4, 0), (-3, 4, 0), (0, 0, 4)]
nppoints shape: (3, 3)
diffs shape: (2, 3)
nppoints shape: (3, 3)
diffs shape: (2, 3)
nppoints shape: (3, 3)
diffs shape: (2, 3)
Complex ? False
filtration value: 5.000000000000001
center [ 4.4408921e-16 -8.8817842e-16  0.0000000e+00]
---- Test for [(0, 5, 0), (3, 4, 0), (-3, 4, 0), (0, 0, 4), (0, 0, -4)]
nppoints shape: (3, 3)
diffs shape: (2, 3)
nppoints shape: (3, 3)
diffs shape: (2, 3)
nppoints shape: (3, 3)
diffs shape: (2, 3)
Complex ? False
filtration value: 5.000000000000001
center [ 4.4408921e-16 -8.8817842e-16  0.0000000e+00]


In [13]:
def task5(points,K,l):
      """ Given a set P of n points in Rd, implement an algorithm that enu-
      merates the simplexes of dimension at most k and filtration value at most l of
      the α-complex and their filtration values.""" 
      enum = enum3(points)
      print(f"enum={enum}")
      filtration_value=0
      IsSimplex = {tuple([i]): 1 for i in range(len(points))}

      simplex = {tuple([i]): Sphere(points[i], 0) for i in range(len(points))} #on initialise le premier simplexe

      for i in range(1,K):
        for j in range(len(enum[i-1])):
              
          current_simplex = enum[i][j]

          for k in range(len(points)):
                  
            if k in current_simplex:
              break
                        
            pn = current_simplex.append(k)

            if not Is_in_alpha_complex(points,[points[idx] for idx in current_simplex]):
              IsSimplex[pn] = 0
              break

            if pn in simplex:
              break

            new_simplex = [points[idx] for idx in current_simplex] + [points[k]]

            MEB = trivial(new_simplex)

            if MEB.radius < l:
              if MEB.radius > filtration_value:
                        filtration_value=MEB.radius
              simplex[pn] = MEB
              IsSimplex[pn] = 1
            else:
              IsSimplex[pn] = 0
              break

            print(f"new alpha-simplex: {pn} -> {MEB.radius} ")
      print(f"filtration value: {filtration_value}")


In [14]:
def test_task5():
 #generate random n points in R^d:
 n=random.randint(5,10)
 print(f"n={n}")

 #d=random.randint(2,5)
 d=3
 print(f"d={d}")


 points=[list(np.random.rand(d)) for i in range(n)]
 print(f"Points: {points}")
 
 #k=random.randint(2,d)
 k=d
 print(f"k={k}")


 l=np.random.rand(0,1)
 l=3


 print(f"l={l}")
 print("----------------Alpha complex et filtration values-----------")
 task5(points,k,l)
 print("----------Simplexes et filtration values--------------")
 task3(points,l)

test_task5()


n=5
d=3
Points: [[0.6631148958368083, 0.9541849040887291, 0.04188422523455548], [0.808534510588786, 0.9099507345068202, 0.3903992868567332], [0.9781141798616209, 0.4727391359018317, 0.7504028231767077], [0.05253546872454751, 0.4313041250107449, 0.9758001379058736], [0.3132480169345282, 0.5263781979641431, 0.9540783783363338]]
k=3
l=3
----------------Alpha complex et filtration values-----------
enum=[[[0], [1], [2], [3], [4]], [[0, 1], [0, 2], [0, 3], [0, 4], [1, 2], [1, 3], [1, 4], [2, 3], [2, 4], [3, 4]], [[0, 1, 2], [0, 1, 3], [0, 1, 4], [0, 2, 3], [0, 2, 4], [0, 3, 4], [1, 2, 3], [1, 2, 4], [1, 3, 4], [2, 3, 4]], [[0, 1, 2, 3], [0, 1, 2, 4], [0, 1, 3, 4], [0, 2, 3, 4], [1, 2, 3, 4]], [[0, 1, 2, 3, 4]]]
nppoints shape: (3, 3)
diffs shape: (2, 3)
nppoints shape: (4, 3)
diffs shape: (3, 3)


LinAlgError: Singular matrix

In [ ]:
def task_visiulization():
    #a completer: creer des points radonnées et les afficher avec les simplexes de l'alpha complexe et leur filtration value et Cech complexe en differentes couleurs
    pass


A finir :
* task_visiulization
* la partie theorique, notamment la partie sur les cellules de voronoi.
* enum est trop couteuse $O(2^n)$ --> l'adapter pour qu'il calcule uniquement les simplexes de taille <= d+1
* dans test_task5(),task3 ne marche pas
* des explications pour chaque questions
* questionnements sur l'utilisation de contains